In [ ]:
import pandas as pd
from pandas import json_normalize
import requests
import numpy as np
import psycopg2

In [ ]:
from sql_queries import drivers_table_insert

#### Drivers data processing

In [ ]:

response_API = requests.get('http://ergast.com/api/f1/drivers.json?limit=855')
print(response_API.status_code)
drivers=response_API.json()
df = json_normalize(drivers['MRData']['DriverTable']['Drivers']) 

In [ ]:
df['permanentNumber'] =df['permanentNumber'].fillna(-99)
df['permanentNumber']=df['permanentNumber'].astype(int)
df['code'] =df['code'].fillna('NO CODE')
df['dateOfBirth'] = pd.to_datetime(df['dateOfBirth'], format="%Y/%m/%d")
for str_col in ['driverId', 'url', 'givenName', 'familyName', 'nationality', 'code'] : 
    df[str_col]= df[str_col].astype('string')

In [ ]:
df.drop('driverId', axis=1, inplace=True)
df=df[['givenName', 'familyName', 'url', 'dateOfBirth', 'nationality', 'permanentNumber' ,'code']]

In [ ]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=f1_api user=postgres password=admin")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)
conn.set_session(autocommit=True)

In [ ]:
for i, row in df.iterrows():
    cur.execute(drivers_table_insert, row.tolist())

In [ ]:
conn.close()